In [2]:
#!/usr/bin/env python
# # -*- coding: utf-8 -*-
#!/usr/bin/env python
#!pip install tensorflow==1.15

import numpy as np
import matplotlib.pyplot as plt
import pickle
import glob, os, inspect
import argparse
import time
import seaborn as sns
# import matplotlib
# matplotlib.use("Qt5Agg")
# #matplotlib.use('Agg')
# from mpl_toolkits.mplot3d import axes3d
# import matplotlib.pyplot as plt
import pandas as pd
import sys,inspect
sys.path.append("C:\\Users\\Vinamr J\\Documents\\qic\\qrc\\hqr\\hqr-master\\chaos\\Postprocess")
sys.path.append("C:\\Users\\Vinamr J\\Documents\\qic\\qrc\\hqr\\hqr-master\\chaos\\Methods\\Models\\Utils")

import utils  # for utils.py
from utils import *
import global_utils  # for global_utils.py

# import utils
# from utils import *
# import sys
# sys.path.append('../Methods/Models/Utils')
# import global_utils

In [3]:
sysname, tidx = "SST", 2
file_path = os.path.abspath("GRU_analysis.ipynb")
current_directory = os.path.dirname(file_path)
mid_directory = os.path.dirname(current_directory)
parent_directory = os.path.dirname(mid_directory)

print("Current Directory:", current_directory)
print("Parent Directory:", parent_directory)

#current_dir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
results_dir = os.path.join(os.path.dirname(mid_directory), "Results")  # Use os.path.join for path concatenation
print(results_dir)

eval_path = os.path.join(results_dir, sysname, 'Evaluation_Data')
print(eval_path)

model_path = os.path.join(results_dir, sysname, 'Trained_Models')
print(model_path)

fig_path = os.path.join(results_dir, sysname, 'Eval_Figures')
if not os.path.isdir(fig_path):
    os.makedirs(fig_path)  # Use os.makedirs to create parent directories if needed
print(fig_path)

Current Directory: c:\Users\Vinamr J\Documents\qic\qrc\hqr\hqr-master\chaos\Data\SST
Parent Directory: c:\Users\Vinamr J\Documents\qic\qrc\hqr\hqr-master\chaos
c:\Users\Vinamr J\Documents\qic\qrc\hqr\hqr-master\chaos\Results
c:\Users\Vinamr J\Documents\qic\qrc\hqr\hqr-master\chaos\Results\SST\Evaluation_Data
c:\Users\Vinamr J\Documents\qic\qrc\hqr\hqr-master\chaos\Results\SST\Trained_Models
c:\Users\Vinamr J\Documents\qic\qrc\hqr\hqr-master\chaos\Results\SST\Eval_Figures


In [5]:
models = []
samples = []
V_values = [5,10,15,20]
alpha_values = [0.0,0.5,0.9]
for V in V_values:
    for alpha in alpha_values:
        string1 = f"hqrc_pinv-RDIM_5-N_used_427-DL_40-Nqr_5-A_{alpha}-J_2.0-fJ_1-V_{V}-NL_0-IPL_300-IUL_0-REG_1e-07-AU_0-NICS_3"
        string2 = f"HQRC-V={V}-alpha={alpha}"
        models.append([os.path.join(eval_path,string1),string2])
        samples.append(string2)
title = "Sea Surface Temperatures"

In [3]:
# models = [[os.path.join(eval_path, "hqrc_pinv-RDIM_5-N_used_427-DL_40-Nqr_5-A_0.9-J_2.0-fJ_1-V_10-NL_0-IPL_20-IUL_0-REG_1e-07-AU_0-NICS_3"),"HQR-5,V-10,DL=40,IPL=20,n_tests=3"],
#           [os.path.join(eval_path, "hqrc_pinv-RDIM_5-N_used_427-DL_8-Nqr_5-A_0.9-J_2.0-fJ_1-V_10-NL_0-IPL_8-IUL_0-REG_1e-07-AU_0-NICS_1"),"HQR-5,V-10,DL=8,IPL=8,n_tests=1"],
#           [os.path.join(eval_path, "hqrc_pinv-RDIM_5-N_used_427-DL_40-Nqr_5-A_0.9-J_2.0-fJ_1-V_10-NL_0-IPL_300-IUL_0-REG_1e-07-AU_0-NICS_1"),"HQR-5,V-10,DL=40,IPL=300,n_tests=1"]]
# samples = ["HQR-5,V-10,DL=40,IPL=20,n_tests=3","HQR-5,V-10,DL=8,IPL=8,n_tests=1","HQR-5,V-10,DL=40,IPL=300,n_tests=1"]
# title = "Sea Surface Temperatures"

In [7]:
rmse_dict = dict()
targets = dict()
outputs = dict()
sp_outputs = dict()
sp_targets = dict()
Wouts = dict()
coeffs = dict()
rmnse_avg_train_dict = dict()
rmnse_avg_test_dict = dict()
for i in range(len(models)):
    rfolder, label = models[i][0], models[i][1]
    fname = os.path.join(rfolder, 'results.pickle')
    if os.path.isfile(fname):
        with open(fname, 'rb') as rfile:
            try:
                rs = pickle.load(rfile)
            except:
                continue
            #print(rs.keys())
            qs = QResults()
            qs.rmnse_avg_test = rs['rmnse_avg_TEST']
            qs.rmnse_avg_train = rs['rmnse_avg_TRAIN']
            qs.n_pred_005_avg_test = rs['num_accurate_pred_005_avg_TEST']
            qs.n_pred_005_avg_train = rs['num_accurate_pred_005_avg_TRAIN']
            qs.n_pred_050_avg_test = rs['num_accurate_pred_050_avg_TEST']
            qs.n_pred_050_avg_train = rs['num_accurate_pred_050_avg_TRAIN']
            qs.model_name = rs['model_name']
            #if qs.rmnse_avg_test != np.inf and qs.rmnse_avg_train != np.inf:
                #print(rs.keys())
            #print(qs.model_name)
            #print('train={}, test={}'.format(qs.rmnse_avg_train, qs.rmnse_avg_test))
            #qs.info()
            rmnse_avg_train_dict[label] = rs['rmnse_avg_TRAIN']
            rmnse_avg_test_dict[label] = rs['rmnse_avg_TEST']
            pred_test = rs['predictions_all_TEST']
            truth_test = rs['truths_all_TEST']
            print(pred_test.shape)
            M = len(pred_test)
            print('{} Number of test'.format(qs.model_name), M)
            rmsels = []
            for j in range(M):
                    rmsels.append(calNRMSE(pred_test[j], truth_test[j]))

            rmse_dict[label] = np.mean(np.array(rmsels), axis=0)
            print(rmse_dict[label].shape)


            targets[label] = truth_test[0]
            outputs[label] = pred_test[0]

            # For frequency
            sp_outputs[label] = rs['sp_pred_TEST']
            sp_targets[label] = rs['sp_true_TEST']
    else:
        print('Not found {}'.format(fname))
    # if trained_models is not None:
    #     rfolder, label = trained_models[i][0], trained_models[i][1]
    #     fname = os.path.join(rfolder, 'data.pickle')
    #     if os.path.isfile(fname):
    #         print('File existed: ', fname)
    #         with open(fname, 'rb') as rfile:
    #             #try:
    #             rs = pickle.load(rfile)
    #             #except:
    #             #    continue
    #             print(rs.keys())
    #             coeffs[label] = np.array(rs['coeffs'])
    #             Wouts[label]  = rs['W_out'][:-1].reshape((coeffs[label].shape[0], -1))
    #             print(Wouts[label].shape, coeffs[label].shape)
    #     else:
    #         print('Not found saved model {}'.format(fname))

(3, 300, 5)
hqrc_pinv-RDIM_5-N_used_427-DL_40-Nqr_5-A_0.0-J_2.0-fJ_1-V_5-NL_0-IPL_300-IUL_0-REG_1e-07-AU_0-NICS_3 Number of test 3
(300,)
(3, 300, 5)
hqrc_pinv-RDIM_5-N_used_427-DL_40-Nqr_5-A_0.5-J_2.0-fJ_1-V_5-NL_0-IPL_300-IUL_0-REG_1e-07-AU_0-NICS_3 Number of test 3
(300,)
(3, 300, 5)
hqrc_pinv-RDIM_5-N_used_427-DL_40-Nqr_5-A_0.9-J_2.0-fJ_1-V_5-NL_0-IPL_300-IUL_0-REG_1e-07-AU_0-NICS_3 Number of test 3
(300,)
(3, 300, 5)
hqrc_pinv-RDIM_5-N_used_427-DL_40-Nqr_5-A_0.0-J_2.0-fJ_1-V_10-NL_0-IPL_300-IUL_0-REG_1e-07-AU_0-NICS_3 Number of test 3
(300,)
(3, 300, 5)
hqrc_pinv-RDIM_5-N_used_427-DL_40-Nqr_5-A_0.5-J_2.0-fJ_1-V_10-NL_0-IPL_300-IUL_0-REG_1e-07-AU_0-NICS_3 Number of test 3
(300,)
(3, 300, 5)
hqrc_pinv-RDIM_5-N_used_427-DL_40-Nqr_5-A_0.9-J_2.0-fJ_1-V_10-NL_0-IPL_300-IUL_0-REG_1e-07-AU_0-NICS_3 Number of test 3
(300,)
(3, 300, 5)
hqrc_pinv-RDIM_5-N_used_427-DL_40-Nqr_5-A_0.0-J_2.0-fJ_1-V_15-NL_0-IPL_300-IUL_0-REG_1e-07-AU_0-NICS_3 Number of test 3
(300,)
(3, 300, 5)
hqrc_pinv-RDIM_5-N

C:\Users\Vinamr J\Documents\qic\qrc\hqr\hqr-master\chaos\Postprocess\utils.py:73: RuntimeWarning: overflow encountered in square
  diff2 = np.square(diff)


In [8]:
model_test_error = []
test_avg_error = []
for label in samples:
    model_test_error.append([label,np.mean(rmse_dict[label])])
    test_avg_error.append(np.mean(rmse_dict[label]))
print(model_test_error)
print("Test Average error = ",np.mean(test_avg_error))

[['HQRC-V=5-alpha=0.0', inf], ['HQRC-V=5-alpha=0.5', inf], ['HQRC-V=5-alpha=0.9', inf], ['HQRC-V=10-alpha=0.0', inf], ['HQRC-V=10-alpha=0.5', inf], ['HQRC-V=10-alpha=0.9', inf], ['HQRC-V=15-alpha=0.0', inf], ['HQRC-V=15-alpha=0.5', inf], ['HQRC-V=15-alpha=0.9', inf], ['HQRC-V=20-alpha=0.0', inf], ['HQRC-V=20-alpha=0.5', inf], ['HQRC-V=20-alpha=0.9', inf]]
Test Average error =  inf


In [10]:
print(rmnse_avg_train_dict)
print(rmnse_avg_test_dict)
error_array_test = []
error_array_train = []
for label in samples:
    error_array_train.append(rmnse_avg_train_dict[label])
    error_array_test.append(rmnse_avg_test_dict[label])
print("Train average error for HQRC =", np.mean(error_array_train))
print("Test average error for HQRC =", np.mean(error_array_test))

{'HQRC-V=5-alpha=0.0': inf, 'HQRC-V=5-alpha=0.5': 0.3081579994254596, 'HQRC-V=5-alpha=0.9': 0.4124705267985448, 'HQRC-V=10-alpha=0.0': inf, 'HQRC-V=10-alpha=0.5': 0.16023250229528269, 'HQRC-V=10-alpha=0.9': inf, 'HQRC-V=15-alpha=0.0': inf, 'HQRC-V=15-alpha=0.5': inf, 'HQRC-V=15-alpha=0.9': 0.45666814371799075, 'HQRC-V=20-alpha=0.0': inf, 'HQRC-V=20-alpha=0.5': inf, 'HQRC-V=20-alpha=0.9': 0.37556120066692283}
{'HQRC-V=5-alpha=0.0': inf, 'HQRC-V=5-alpha=0.5': inf, 'HQRC-V=5-alpha=0.9': inf, 'HQRC-V=10-alpha=0.0': inf, 'HQRC-V=10-alpha=0.5': inf, 'HQRC-V=10-alpha=0.9': inf, 'HQRC-V=15-alpha=0.0': inf, 'HQRC-V=15-alpha=0.5': inf, 'HQRC-V=15-alpha=0.9': inf, 'HQRC-V=20-alpha=0.0': inf, 'HQRC-V=20-alpha=0.5': inf, 'HQRC-V=20-alpha=0.9': inf}
Train average error for HQRC = inf
Test average error for HQRC = inf


In [11]:
models = []
samples = []
V_values = [5,10,15,20]
alpha_values = [0.0,0.5,0.9]
for V in V_values:
    for alpha in alpha_values:
        string1 = f"hqrc_pinv-RDIM_5-N_used_427-DL_40-Nqr_5-A_{alpha}-J_2.0-fJ_1-V_{V}-NL_0-IPL_300-IUL_0-REG_1e-07-AU_0-NICS_3"
        string2 = f"HQRC-V={V}-alpha={alpha}"
        models.append([os.path.join(model_path,string1),string2])
        samples.append(string2)
title = "Sea Surface Temperatures"

In [12]:
train_time_dict = dict()
memory_dict = dict()
trainable_parameters_dict = dict()
for i in range(len(models)):
    rfolder, label = models[i][0], models[i][1]
    fname = os.path.join(rfolder, 'data.pickle')
    if os.path.isfile(fname):
        with open(fname, 'rb') as rfile:
            try:
                rs = pickle.load(rfile)
            except:
                continue
            #print(rs.keys())
            train_time_dict[label]=rs["total_training_time"]
            trainable_parameters_dict[label]=rs["n_trainable_parameters"]
            memory_dict[label]=rs["memory"]
    else:
        print('Not found {}'.format(fname))

In [13]:
train_time_dict

{'HQRC-V=5-alpha=0.0': 6.418042182922363,
 'HQRC-V=5-alpha=0.5': 6.617361783981323,
 'HQRC-V=5-alpha=0.9': 8.417607307434082,
 'HQRC-V=10-alpha=0.0': 15.486985921859741,
 'HQRC-V=10-alpha=0.5': 15.717888832092285,
 'HQRC-V=10-alpha=0.9': 15.289921045303345,
 'HQRC-V=15-alpha=0.0': 24.318379402160645,
 'HQRC-V=15-alpha=0.5': 24.49633502960205,
 'HQRC-V=15-alpha=0.9': 24.94126844406128,
 'HQRC-V=20-alpha=0.0': 35.273587226867676,
 'HQRC-V=20-alpha=0.5': 31.323994636535645,
 'HQRC-V=20-alpha=0.9': 32.06458377838135}

In [14]:
memory_dict

{'HQRC-V=5-alpha=0.0': 102.58984375,
 'HQRC-V=5-alpha=0.5': 102.36328125,
 'HQRC-V=5-alpha=0.9': 102.2265625,
 'HQRC-V=10-alpha=0.0': 104.94921875,
 'HQRC-V=10-alpha=0.5': 104.72265625,
 'HQRC-V=10-alpha=0.9': 105.3125,
 'HQRC-V=15-alpha=0.0': 108.953125,
 'HQRC-V=15-alpha=0.5': 109.11328125,
 'HQRC-V=15-alpha=0.9': 109.1953125,
 'HQRC-V=20-alpha=0.0': 114.30078125,
 'HQRC-V=20-alpha=0.5': 114.265625,
 'HQRC-V=20-alpha=0.9': 112.6640625}

In [15]:
trainable_parameters_dict

{'HQRC-V=5-alpha=0.0': 755,
 'HQRC-V=5-alpha=0.5': 755,
 'HQRC-V=5-alpha=0.9': 755,
 'HQRC-V=10-alpha=0.0': 1505,
 'HQRC-V=10-alpha=0.5': 1505,
 'HQRC-V=10-alpha=0.9': 1505,
 'HQRC-V=15-alpha=0.0': 2255,
 'HQRC-V=15-alpha=0.5': 2255,
 'HQRC-V=15-alpha=0.9': 2255,
 'HQRC-V=20-alpha=0.0': 3005,
 'HQRC-V=20-alpha=0.5': 3005,
 'HQRC-V=20-alpha=0.9': 3005}